# Cats and dogs clasificator

Details

* Binary Classification problem
* Colored Images
* No Standard Dimension
* Kaggle Dataset > https://drive.google.com/open?id=16AlwTDOeyFaiP3RPxKOk5s80IycK80X4
* We will use subset of this data 



Data

Train (2000 images)
*   class_a (1000 images)
*   class_b (1000 images)
---
Valid
*   class_a (500 images)
*   class_b (500 images)
---
Test
*   Test_folder (1000 images)


In [ ]:
# libraries
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras